# intro

## Autores

Juan Pablo Jaramillo Tobon CC 1216727112

Daniel Alejandro Higuita Usuga CC 1152706601


## Dataset

Los registros de viajes en taxi amarillo y verde incluyen campos que capturan fechas/horas de recogida y devolución, lugares de recogida y devolución, distancias de viaje, tarifas detalladas, tipos de tarifas, tipos de pago y recuentos de pasajeros informados por el conductor. Los datos utilizados en los conjuntos de datos adjuntos se recopilaron y proporcionaron a la Comisión de Taxis y Limusinas de Nueva York (TLC).

El dataset posee 19 columnas de las cuales usaremos solo 4

* tpep_pickup_datetime: fecha y hora de inicio de carrera
* pickup_longitude: longitud geoespacial de recogida de la carrera
* pickup_latitude: latitud geoespacial de recogida de la carrera
* tip_amount: propina dejada

## Objetivo

El objetivo es la creacion de un modelo que permita la prediccion de la propina dejada por los usuarios dependiendo de el lugar y la hora donde se recoge la propina

# Dependencias

In [ ]:
# Carga de env variables 
# ==============================================================================
import yaml
with open('config.yaml') as f:
    env_vars = yaml.safe_load(f)

## Librerias

In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np


# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib
import os


# Gestion de librerias
# ==============================================================================
from importlib import reload


# Matemáticas y estadísticas
# ==============================================================================
import math


# Preparación de datos
# ==============================================================================
from sklearn.neighbors import LocalOutlierFactor

#Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split


#Escalar Variables
from sklearn.preprocessing import MinMaxScaler


#Evaluación del modelo
from sklearn import metrics
#from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve


#Creación de modelo
from sklearn import svm


#configuracion de hiperparámetros
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

## Datasets

In [ ]:
csv2015_1 = pd.read_csv('data/yellow_tripdata_2015-01.csv')
csv2015_2 = pd.read_csv('data/yellow_tripdata_2015-02.csv')
csv2015_3 = pd.read_csv('data/yellow_tripdata_2015-03.csv')

df = pd.concat([csv2015_1,csv2015_2,csv2015_3],ignore_index=True)

## Preparacion de Datos

### Eliminación de Columnas innecesarias

In [ ]:
df = df[["tpep_pickup_datetime","pickup_longitude","pickup_latitude","tip_amount"]]

### Minimización del Dataset

In [ ]:
if env_vars['minimize_dataset']:
    particion = (100-0.1)/100
    X_train, X_test, y_train, y_test = train_test_split(df['reseña'],df['clasificacion'], test_size=particion, random_state=42)
    df = pd.concat([y_train, X_train], axis=1)
    del X_train, X_test, y_train, y_test 

### Revisión de Duplicados

In [ ]:
df.loc[df.duplicated()]

### Eliminación de Duplicados

In [ ]:
df = df.drop_duplicates()

### Información del Dataset

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.groupby('clasificacion').count()

## Preprocesamiento y normalizacion

In [ ]:
# extraer del timestap minuto del dia, dia de la semana, mes del año

# "pickup_longitude","pickup_latitude","tip_amount"

df["dia"] = df["tpep_pickup_datetime"].dt.dayofweek
df["mes"] = df["tpep_pickup_datetime"].dt.month
df["minuto"] = df["tpep_pickup_datetime"].dt.minute + (df["tpep_pickup_datetime"].dt.hour*60)

df.drop(columns=['tpep_pickup_datetime'],inplace=True)



In [ ]:
day_scaler = MinMaxScaler()
month_scaler = MinMaxScaler()
minute_scaler = MinMaxScaler()
longitude_scaler = MinMaxScaler()
latitude_scaler = MinMaxScaler()
tip_scaler = MinMaxScaler()

day_scaler.fit(df["dia"])
month_scaler.fit(df["mes"])
minute_scaler.fit(df["minuto"])
longitude_scaler.fit(df["pickup_longitude"])
latitude_scaler.fit(df["pickup_latitude"])
tip_scaler.fit(df["tip_amount"])


